# 環境構築

In [1]:
!pip install gensim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## データセットのロードと学習準備

See:
  - [Define a Function to Read and Preprocess Text | gensim Doc
](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html?highlight=dataset#define-a-function-to-read-and-preprocess-text)

The file we’re reading is a **corpus**. Each line of the file is a **document**.

In [4]:
# import libs
import json
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import smart_open

In [5]:
root_dir_path = '/content/drive/My Drive/00-Biz-Dev/portfolio-word-embedding/'

In [6]:
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="utf-8") as f:
        dataset = json.load(f)
        for doc in dataset:
          doc_tag = int(doc['tag'])
          doc_text = doc['text']
          if tokens_only:
            yield doc_text
          else:
            # For training data, add tags
            yield TaggedDocument(doc_text, [doc_tag])

## 学習

See:
  - [Doc2Vecによる文書ベクトル推論の安定化について | Sansan Builders Blog](https://buildersbox.corp-sansan.com/entry/2019/04/10/110000#Doc2Vec%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E6%8E%A8%E8%AB%96%E3%81%AE%E5%95%8F%E9%A1%8C%E7%82%B9)

In [7]:
train_corpus = list(read_corpus(root_dir_path + 'train_dataset.json'))
test_corpus = list(read_corpus(root_dir_path + 'test_dataset.json', tokens_only=True))

In [8]:
# see coupus
train_corpus[:10]

[TaggedDocument(words=['参加', 'し', 'たい', 'けど', '睡眠', '時間', 'が', '優先', 'ェ・・・', '＿|￣|○'], tags=[0]),
 TaggedDocument(words=['そう', 'です', 'ねー', '、', '平日', 'PV', '下がっ', 'てる', '分', 'トータル', 'も', '減っ', 'て', 'ます', 'ね', '。', 'あ', '、', 'ちなみに', '土日', 'は', 'コロナ', '前', 'と', 'ほぼ', '変わら', 'ない', 'っす', '。'], tags=[1]),
 TaggedDocument(words=['・', 'AmplitudePhase', '2', '・', '英語', '学習', '進行', '中', '・', '引っ越し', '関係', '引き続き'], tags=[2]),
 TaggedDocument(words=['おごる', '金', 'なし'], tags=[3]),
 TaggedDocument(words=['さん', '先', 'の', '回答', 'に', 'も', '書か', 'れ', 'て', 'い', 'まし', 'た', 'ね', '、', 'すみません'], tags=[4]),
 TaggedDocument(words=['その', 'ため', 'の', '流れ', 'を', '作っ', 'た', '感', 'あり', 'ます', 'から', 'ね', 'ｗ', 'ちゃんと', 'ポートフォリオ', '作成', '用', 'の', 'コンテンツ', 'と', 'す', 'べく', '画策', '中', 'です', '♪'], tags=[5]),
 TaggedDocument(words=['まさかの', 'Wish', 'リスト', 'が', '上位', 'wwww'], tags=[6]),
 TaggedDocument(words=['オンライン', '参加', 'で', 'お願い', 'いたし', 'ます'], tags=[7]),
 TaggedDocument(words=['あ', '、', 'そう', 'これ', 'です', '！', 'こんな', '感じ',

In [9]:
model = Doc2Vec(vector_size=300, min_count=1, epochs=40)
model

In [10]:
model.build_vocab(train_corpus)

In [11]:
model.corpus_count

24998

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
v = model.infer_vector(['私', 'は', '山田', '孝之', 'です', '。'])
len(v)

In [ ]:
# save
import pickle
with open(root_dir_path + 'trained_doc2vec.model', 'wb') as f:
    pickle.dump(model, f)